In [25]:
# import relevant libs
import plotly.express as px
import pandas as pd

In [26]:
# import metric files from the optuna trial of concern
df_metrics = pd.read_parquet('/Users/thomas/Repositories/MasterThesis30313/output/metrics/20241105_trial_7_DDQAugmentedNoisyLSTMNN_metrics.pq')
df_q_metrics = pd.read_parquet('/Users/thomas/Repositories/MasterThesis30313/output/metrics/20241105_trial_7_DDQAugmentedNoisyLSTMNN_q_metrics.pq')
df_action_metrics = pd.read_parquet('/Users/thomas/Repositories/MasterThesis30313/output/metrics/20241105_trial_7_DDQAugmentedNoisyLSTMNN_action_logs.pq')  

In [27]:
# pivot standard metrics prior to compare episodes
df_metrics_pivot = pd.melt(df_metrics, id_vars=['episode'], var_name='value_type', value_name='value').sort_values(['episode', 'value_type']).reset_index(drop=True)

In [28]:
# moving average loss/reward per episode
fig = px.line(df_metrics_pivot[df_metrics_pivot['value_type'].isin(['mvg_avg_score', 'mvg_avg_loss'])], x="episode", y="value", color='value_type', title='Moving Average Loss/Reward per Episode', log_y=True)

fig.update_layout(width=800, height=600)

fig.show()

In [29]:
# Aggregate Actions per episode Information
actions_per_episode = df_action_metrics.groupby(['episode', 'action']).agg(
    action_count=('action', 'size'),
    q_values_sum=('q_values_sum', 'sum'),
    q_values_mean=('q_values_sum', 'mean'),
    q_values_median=('q_values_sum', 'median')
).reset_index()

# Total no. of actions per episode
actions_per_episode_sum = actions_per_episode.groupby(['episode'])['action_count'].sum().reset_index()

# Grouping data by episode and action to get the total count of each action per episode
actions_per_episode_grouped = actions_per_episode.groupby(['episode', 'action'])['action_count'].sum().reset_index()

# Pivot the data to have each action as a separate column for easier plotting
actions_per_episode_pivot = actions_per_episode_grouped.pivot(index='episode', columns='action', values='action_count').fillna(0)

In [30]:
# Total number of actions per episode
# Create a line chart with one line for each action
fig = px.line(actions_per_episode_sum, 
              x=actions_per_episode_sum['episode'], 
              y=actions_per_episode_sum['action_count'],
              labels={'value': 'Action Count', 'episode': 'Episode'},
              title='Action Counts per Episode')

# Update the layout for better visibility
fig.update_layout(
    yaxis_title='Number of Actions',
    xaxis_title='Episode',
    legend_title_text='Sum of Actions',
    hovermode="x unified", 
    width=800, 
    height=600
)

fig.show()

In [31]:
# Create a line chart with one line for each action
fig = px.line(actions_per_episode_pivot, 
              x=actions_per_episode_pivot.index, 
              y=actions_per_episode_pivot.columns,
              labels={'value': 'Action Count', 'episode': 'Episode'},
              title='Action Counts per Episode Differentiated by Action')

# Update the layout for better visibility
fig.update_layout(
    yaxis_title='Number of Actions',
    xaxis_title='Episode',
    legend_title_text='Action',
    hovermode="x unified", 
    width=800, 
    height=600
)

fig.show()

In [32]:
# Distribution of Q-values Across Actions
fig = px.histogram(actions_per_episode, x='q_values_mean',
                    title='Distribution of Q-values Across Actions',
                    labels={'q_values_mean': 'Mean Q-value', 'count': 'Frequency'},
                    marginal="box",  # Adds a boxplot for better visualization
                    color_discrete_sequence=['#00CC96'])

fig.update_layout(width=800, height=600)

fig.show()

In [33]:
# Distribution of Q-values per Action
fig = px.box(actions_per_episode, x='action', y='q_values_mean',
              title='Distribution of Q-values per Action',
              labels={'action': 'Action', 'q_values_mean': 'Mean Q-value'},
              color='action',  # Color by action for better distinction
              color_discrete_sequence=px.colors.qualitative.Set3)

fig.update_layout(width=800, height=600)

fig.show()

In [34]:
# Grouping data by episode and action to get the total sum of Q-values per action per episode
q_values_sum_grouped = actions_per_episode.groupby(['episode', 'action'])['q_values_sum'].sum().reset_index()

# Pivot the data to have each action as a separate column for easier plotting
q_values_sum_pivot = q_values_sum_grouped.pivot(index='episode', columns='action', values='q_values_sum').fillna(0)

# Create a line chart with one line for each action, showing the Q-value sum over episodes
fig = px.line(q_values_sum_pivot, 
              x=q_values_sum_pivot.index, 
              y=q_values_sum_pivot.columns,
              labels={'value': 'Q-value Sum', 'episode': 'Episode'},
              title='Q-value Sums per Episode Differentiated by Action')

# Update the layout for better visibility
fig.update_layout(
    yaxis_title='Q-value Sum',
    xaxis_title='Episode',
    legend_title_text='Action',
    hovermode="x unified", 
    width=800, 
    height=600
)

fig.show()